<a href="https://colab.research.google.com/github/LotusBro98/Lanat2019/blob/master/Machine_Learning_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинное обучение
## Идентификация объектов

---

<img width="100%" src="https://i.pinimg.com/736x/83/d2/14/83d214b253298e3eb191d071921d7cc1.jpg" />

---

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Загружаем датасет

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
N_CLASSES = len(class_names)

Визуализируем датасет

In [0]:
for i in range(3):
  plt.figure()
  plt.imshow(train_images[i])
  plt.colorbar()
  plt.grid(False)
  plt.show()

  print(train_labels[i], ": ", class_names[train_labels[i]])
  print()

Приводим к виду, пригодному для обучения

In [0]:
# Масштабируем от -1 до 1
X_train = train_images / 127.5 - 1
# Добавляем размерность для цветовых каналов (у нас он только 1)
X_train = tf.expand_dims(X_train, axis=-1)

# каждое число заменяем вектором, в котором все нули, кроме указанного
Y_train = tf.one_hot(train_labels, N_CLASSES)

X_test = test_images / 127.5 - 1
X_test = tf.expand_dims(X_test, axis=-1)
X_test = tf.cast(X_test, tf.float32)
Y_test = tf.one_hot(train_labels, N_CLASSES)

# Нарисуем первые 3 элемента для наглядности
for i in range(3):
  plt.imshow(X_train[i,:,:,0])
  plt.colorbar()
  plt.grid(False)
  plt.show()
  
  print(Y_train[i].numpy())
  print(class_names[np.argmax(Y_train[i].numpy())])
  print()

Датасет готов

## Построим модель сверточной нейросети для классификации (LeNet)

In [0]:
def build_LeNet():
  inputs = tf.keras.layers.Input(shape=[28, 28, 1])
  x = inputs

  x = tf.keras.layers.Convolution2D(64, 3)(x)
  x = tf.keras.layers.MaxPooling2D(2, 2)(x)

  x = tf.keras.layers.Convolution2D(32, 3)(x)
  x = tf.keras.layers.MaxPooling2D(2, 2)(x)

  x = tf.keras.layers.Flatten()(x)
  
  x = tf.keras.layers.Dense(200, activation='relu')(x)
  x = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(x)

  model = tf.keras.Model(inputs = inputs, outputs = x)
  return model


model = build_LeNet()

Выберем лосс-функцию и оптимизатор и подготовим модель к обучению.

In [0]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

Запустим обучение

In [0]:
model.fit(X_train, Y_train, epochs=20, batch_size=100)

## Проверим обученную нейросеть

In [0]:
ind = np.random.randint(0, len(test_images))
img = X_test[ind:ind+1]
label_real = test_labels[ind]

# Рисуем картинку
plt.imshow(img[0, :, :, 0])
plt.colorbar()
plt.show()

# Нейросеть делает предсказание
predictions = model.predict(img)[0]

# Выбираем элемент, в котором она наиболее уверена
label_predicted = np.argmax(predictions)
name_predicted = class_names[label_predicted]

print("I think tis is... " + name_predicted + "!")

print("~~~~~~~~~~~~~~~")

for name, score in zip(class_names, predictions):
  print("%s: %.3f" % (name, score))
  
print("~~~~~~~~~~~~~~~")
print("Label says it is " + class_names[label_real])